## Product Recommender System

In [28]:
import pandas as pd
import numpy as np
import nltk 

In [29]:
amazon=pd.read_csv('amazon_product.csv')

In [30]:
amazon.head()

,id,Title,Description,Category
0,1,Swissmar Capstore Select Storage Rack for 18-...,Swissmar's capstore select 18 storage unit kee...,Home & Kitchen Kitchen & Dining Kitchen Utens...
1,2,Gemini200 Delta CV-880 Gold Crown Livery Airc...,Welcome to the exciting world of GeminiJets! O...,Toys & Games Hobbies Models & Model Kits Pre-...
2,5,Superior Threads 10501-2172 Magnifico Cream P...,"For quilting and embroidery, this product is m...","Arts, Crafts & Sewing Sewing Thread & Floss S..."
3,6,Fashion Angels Color Rox Hair Chox Kit,Experiment with the haute trend of hair chalki...,Beauty & Personal Care Hair Care Hair Colorin...
4,8,Union Creative Giant Killing Figure 05: Daisu...,From Union Creative. Turn your display shelf i...,Toys & Games › Action Figures & Statues › Sta...


In [31]:
amazon.drop(columns='id',inplace=True)

In [32]:
amazon.head()

,Title,Description,Category
0,Swissmar Capstore Select Storage Rack for 18-...,Swissmar's capstore select 18 storage unit kee...,Home & Kitchen Kitchen & Dining Kitchen Utens...
1,Gemini200 Delta CV-880 Gold Crown Livery Airc...,Welcome to the exciting world of GeminiJets! O...,Toys & Games Hobbies Models & Model Kits Pre-...
2,Superior Threads 10501-2172 Magnifico Cream P...,"For quilting and embroidery, this product is m...","Arts, Crafts & Sewing Sewing Thread & Floss S..."
3,Fashion Angels Color Rox Hair Chox Kit,Experiment with the haute trend of hair chalki...,Beauty & Personal Care Hair Care Hair Colorin...
4,Union Creative Giant Killing Figure 05: Daisu...,From Union Creative. Turn your display shelf i...,Toys & Games › Action Figures & Statues › Sta...


In [33]:
amazon.isnull().sum()

Title          0
Description    0
Category       0
dtype: int64

In [34]:
from nltk.stem.snowball import SnowballStemmer
stemmer=SnowballStemmer('english')
def tokenize_stem(text):
    tokens = nltk.word_tokenize(text.lower())
    stemmed =[ stemmer.stem(w) for w in tokens ]
    return ' '.join(stemmed)

In [35]:
amazon['stemmed_tokens']=amazon.apply(lambda row:tokenize_stem(row['Title']+' '+ row['Description']),axis=1)

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [44]:
tfidfv=TfidfVectorizer(tokenizer=tokenize_stem)
def cosine_sim(txt1,txt2):
    matrix=tfidfv.fit_transform([txt1,txt2])
    return cosine_similarity(matrix)[0][1]

In [45]:
def search_product(query):
    stemmed_query=tokenize_stem(query)
    # calculating cosine similarity between query and stemmed tokens columns
    amazon['similarity']=amazon['stemmed_tokens'].apply(lambda x:cosine_sim(stemmed_query,x))
    res=amazon.sort_values(by=['similarity'],ascending=False).head(10)[['Title','Description','Category']]
    return res

In [46]:
search_product(' Swissmar Capstore Select Storage Rack for 18-Pack')

,Title,Description,Category
475,Pacon Spectra Glitter Sparkling Crystals,"Change all to: Spectra Glitter, Sparkling Crys...","Arts, Crafts & Sewing › Crafting › Craft Supp..."
463,Versio Mobile 3-Pack Screen Protector for LG ...,Clear 3 Pack Screen Protector For LG G2,Cell Phones & Accessories Accessories Screen ...
593,Qmadix QM-SPHTC6410 Screen Protector for HTC ...,"Protect your device's display from dust, scrat...",Cell Phones & Accessories Accessories Screen ...
526,"Titan Classic Spandex Dreadlock Cap, Regular",Spandex Dreadlock Cap - Regular Size. Covers M...,Beauty & Personal Care › Hair Care
177,Sargent Art 22-7099 6-Count 8-Ounce Fluoresce...,Sargent art's washable watercolor magic is uni...,"Arts, Crafts & Sewing › Painting, Drawing & A..."
0,Swissmar Capstore Select Storage Rack for 18-...,Swissmar's capstore select 18 storage unit kee...,Home & Kitchen Kitchen & Dining Kitchen Utens...
147,Martha Stewart Crafts 32802 Stencil Brushes (...,Transfer patterns with ease by painting over t...,"Arts, Crafts & Sewing Crafting Paper & Paper ..."
110,Core Products Elastic Criss Cross Back Suppor...,The Elastic Crisscross Back Support helps reli...,Health & Household › Medical Supplies & Equip...
384,Visol Ferrara Red Cigar Case - Holds 2-3 Cigars,"Vibrant red, soft leather wraps this crushproo...",Health & Household Household Supplies Tobacco...
13,BRUT After Shave Classic Fragrance 5 oz (Pack...,BRUT After Shave Classic Fragrance 5 oz (Pack ...,Beauty & Personal Care › Shave & Hair Removal...
